Aqui é para testar se o dispositivo está disponível para usar a GPU.
E import as bibliotecas necessárias para rodar o código.
Fizemos assim para também podermos rodar o código localmente, e no Google Colab.

In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import torch

Já que juntamos todos os dados em um CSV com duas colunas: statement do problema E editorial;
a gente tem que utilizar uma biblioteca específica, com a função load_dataset para carregar o arquivo.

In [2]:
from datasets import load_dataset

Aqui é só pra saber qual é o dispositivo que está disponível para usar a GPU.
No meu computador tive de rodar com o CPU porque minha AMD GPU não é compatível com o TensorFlow ROCM.

In [3]:
device_name = torch.cuda.get_device_name(0)

torch_device_name = "cpu" if "AMD Radeon RX 580 2048SP" in device_name else "cuda"

print(f"Arquitetura de dispositivo: '{device_name}' e nome do dispositivo: '{torch_device_name}'" )

Arquitetura de dispositivo: 'AMD Radeon RX 580 2048SP' e nome do dispositivo: 'cpu'


In [ ]:
device = torch.device(torch_device_name)

Rodamos o modelo inicial que decidimos usar, que é o t5-small, que foi construído 
com o intuito de ser um modelo de tradução de texto, mas que pode ser utilizado para
outras tarefas de NLP.

In [5]:
model_name = "t5-small"
model = T5ForConditionalGeneration.from_pretrained(model_name)
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
# Aqui é para carregar o tokenizador do modelo que vamos usar, que é o t5-small.
tokenizer = T5Tokenizer.from_pretrained(model_name)